In [ ]:
!pip install gdown

!gdown --id 1kbYmGkMsQGVBdnDStxx3nz79WNh5JXy7 # training_set_clean_only_text.txt
!gdown --id 1XP30I7gKxhY1jOCOFlcIWDQDnZ3Qk9rZ # training_set_clean_only_tags.txt
!gdown --id 1z0Laz8jCGR-GQcMNL2_qKsGnbL4VRRCY # test_set_clean_only_text.txt
!gdown --id 1o_XRCnKlScCrZRk5suzjwzljT7YonQ2q # test_set_clean_only_tags.txt


!gdown --id 1gJL8GYTuWj5BXUUFnCy--AE4xEZlDr1u # corpora

!gdown --id 1-IeRYiTy5rujiTg97jrEzhCSpdJkoBL1 # dataset4

!gdown --id 1IezKpxY0c8OHv0nq44yrUH8pYtHnsnY- # dataset5


## data processing

In [ ]:
!pip install morfeusz2
import morfeusz2
morf = morfeusz2.Morfeusz()
import pandas as pd
import numpy as np
import re

import string
from string import ascii_lowercase
from sklearn.metrics import f1_score, confusion_matrix

from gensim.models import Word2Vec

!pip install emoji
import emoji
from string import punctuation

In [ ]:
def read_file(file_name): # read given file and create list of phrases
  lines = []
  with open(file_name, encoding="UTF-8-SIG") as text:
    for line in text:
      lines.append(line[:-1])
  return lines

  # merge texts with labels
def merge_lists(list1, list2):
  merged = []
  for i in range(len(list1)):
    merged.append([list1[i], list2[i]])
  return merged

text = read_file('training_set_clean_only_text.txt')
labels = read_file('training_set_clean_only_tags.txt')

texts_labels = merge_lists(text, labels)

text_test = read_file('test_set_clean_only_text.txt')
labels_test = read_file('test_set_clean_only_tags.txt')

texts_labels_test = merge_lists(text_test, labels_test)

In [ ]:
not_wanted_words = ['@anonymized_account']

def word_process(word):
  if word in not_wanted_words:
    return False

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
  word = emoji_pattern.sub(r'', word) # no emoji
  word = word.lower()
  word = word.translate(str.maketrans('', '', string.punctuation))
  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break 
  word = word.lower()
  if len(word)==0:
    return False
  return word

def processing_v2(texts, not_wanted_words): # create corpuse for hate, normal and both
  k=0
  corpus_normal = {}
  corpus_hate = {}
  corpus_whole = {}
  normal_num = 0
  hate_num = 0

  text_i = 0
  for text in texts:
    text_i +=1

    words = text[0].split()
    label = text[1]
    if words[0] == 'RT': 
      continue
    else:
      for word in words:
      
        word = word_process(word)
        if word == False: continue
        if word not in corpus_whole:
          corpus_whole[word] = 1
        else:
          corpus_whole[word] = corpus_whole[word] + 1

        if label == '0':
          if word not in corpus_normal:
            corpus_normal[word] = 1
          else:
            corpus_normal[word] = corpus_normal[word] + 1            
        if label == '1':
          if word not in corpus_hate:
            corpus_hate[word] = 1
          else:
            corpus_hate[word] = corpus_hate[word] + 1

  return corpus_whole, corpus_hate, corpus_normal


corpus_whole_v2, corpus_hate_v2, corpus_normal_v2 = processing_v2(texts_labels, not_wanted_words)

In [ ]:
def clean_dataset(dataset, processed_words):
  dataset_clean = []
  for line in dataset:
    row = []
    words = line[0].split()
    if words[0] == 'RT': 
      continue
    for word in words:
      if word in processed_words:
        word = processed_words[word]
      else:
        new_word = word_process(word)
        processed_words[word] = new_word
        word = new_word
      if word != False: 
        row.append(word)
    dataset_clean.append(' '.join(row))
  return dataset_clean

def clean_dataset_corpus(line):
  words = line.split()
  if words[0] == 'RT': 
    return ''
  row = []
  for word in words:
    word = word_process(word)
    if word != False: 
      row.append(word)
  return row

In [ ]:
def split_for_embeddings(dataset):
  return [ text.split() for text in dataset]

In [ ]:
clean_train = clean_dataset(texts_labels)
clean_test = clean_dataset(texts_labels_test)

In [ ]:
clean_train_tokenized = split_for_embeddings(clean_train)
clean_test_tokenized = split_for_embeddings(clean_test)

In [ ]:
clean_train

['dla ja faworyt do tytuł być cracovia zobaczyć czy typ się sprawdzić',
 'brawo ty daria kibic mój bycie na dobry i zły',
 'super polski premier składać kwiat na grób kolaborant ale doczekać czas',
 'musieć inny drogi nie mama',
 'odrzut natychmiastowy kwaśny mina mamić problem',
 'jaki on być fajny xdd pamiętać że spóźnić się na on pierwszy zająć i to sporo i za kara kazać ja usiąść w pierwszy ławka xd',
 'no nie mój u my szczęście',
 'dawno kto tak wredny nie widzieć xd',
 'zaległość były ale ważny czy były wezwanie do zapłata z który się klub nie wywiązać',
 'gdzie być brudziński być kłamca i marny kutas',
 'no mamić nadziać',
 'może gustować w starszy pani',
 'zostawić tama 3 latać temu notatka na mankiet musieć się wrócić',
 'oprawa do krzyś metr księga_psalmów pinokio n móc bycie gangster a zostać 3 ligowy frajer',
 'znowu być komunikat o polski pirata drogowy w słowacja',
 'mówić człowiek który chcieć nałożyć sankcja na polska gratulacje niepodlegla 11lisopada',
 'o jaki zachowa

## embeddings only training data

In [ ]:
model_embeddings100 = Word2Vec(sentences=clean_train_tokenized, size=100, window=5, min_count=1, workers=4)


In [ ]:
model_embeddings100.wv.most_similar('mówić', topn=10) # not working

[('on', 0.9999614953994751),
 ('ja', 0.999959409236908),
 ('a', 0.9999592304229736),
 ('chyba', 0.9999585151672363),
 ('być', 0.9999569654464722),
 ('tylko', 0.9999567270278931),
 ('już', 0.9999544620513916),
 ('bo', 0.9999532699584961),
 ('tu', 0.9999532699584961),
 ('taki', 0.999953031539917)]

## cleaning - polish corpora

In [ ]:
def clean_dataset_from_file():
  dataset_clean = []
  processed_words = {}
  # k=0
  with open('polish_corpora.txt', 'r', encoding = 'utf8') as file:
    with open('dataset_clean4.txt', 'a') as f:

      for i, line in enumerate(file):
        if i%10000==0: 
          print(i)
          for item in dataset_clean:
              f.write(item + '\n')

          dataset_clean = []
        if i==5000000: break
        row = []
        words = line.split()
        if words[0] == 'RT': 
          continue
        for word in words:
          if word in processed_words:
            word = processed_words[word]
          else:
            new_word = word_process(word)
            processed_words[word] = new_word
            word = new_word
          if word != False: 
            row.append(word)

        dataset_clean.append(' '.join(row))
    return dataset_clean, processed_words
dataset_clean, processed_words = clean_dataset_from_file()

In [ ]:
dataset_clean = []
with open('dataset_clean4.txt', 'r') as file2:
  for i, line in enumerate(file2):
    dataset_clean.append(line.split())

In [ ]:
len(dataset_clean)

4999998

## clean polish corpora -better processing

In [ ]:
emojis_to_words = {'CONFOUNDED FACE': 'zmieszanie',
                  'CRYING FACE': 'płacz',
                  'DISAPPOINTED BUT RELIEVED FACE': 'rozczarowanie',
                  'DISAPPOINTED FACE': 'rozczarowanie',
                  'EXPRESSIONLESS FACE': 'bezwyrazowy',
                  'FACE SAVOURING DELICIOUS FOOD': 'pyszności',
                  'FACE SCREAMING IN FEAR': 'strach',
                  'FACE THROWING A KISS': 'pocałunek',
                  'FACE WITH COLD SWEAT': 'stres',
                  'FACE WITH MEDICAL MASK': 'maska',
                  'FACE WITH OK GESTURE': 'ok',
                  'FACE WITH OPEN MOUTH': 'zdziwienie',
                  'FACE WITH STUCK-OUT TONGUE': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND WINKING EYE': 'język',
                  'FACE WITH TEARS OF JOY': 'radość',
                  'FEARFUL FACE': 'strach',
                  'FLUSHED FACE': 'zaczerwienienie',
                  'GRIMACING FACE': 'grymas',
                  'GRINNING FACE': 'uśmiech',
                  'GRINNING FACE WITH SMILING EYES': 'uśmiech',
                  'HAPPY PERSON RAISING ONE HAND': 'radość',
                  'HEAR-NO-EVIL MONKEY': 'małpka',
                  'HUSHED FACE': 'uciszenie',
                  'LOUDLY CRYING FACE': 'płacz',
                  'NEUTRAL FACE': 'neutralność',
                  'PENSIVE FACE': 'zamyślenie',
                  'PERSEVERING FACE': 'wytrwałość',
                  'PERSON BOWING DEEPLY': 'ukłon',
                  'PERSON RAISING BOTH HANDS IN CELEBRATION': 'radość',
                  'PERSON WITH FOLDED HANDS': 'ok',
                  'POUTING FACE': 'dąsanie',
                  'SEE-NO-EVIL MONKEY': 'małpka',
                  'SLIGHTLY SMILING FACE': 'uśmiech',
                  'SMILING FACE WITH HALO': 'uśmiech',
                  'SMILING FACE WITH HEART-SHAPED EYES': 'miłość',
                  'SMILING FACE WITH OPEN MOUTH': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND COLD SWEAT': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES': 'uśmiech',
                  'SMILING FACE WITH SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH SUNGLASSES': 'uśmiech',
                  'SPEAK-NO-EVIL MONKEY': 'małpka',
                  'UNAMUSED FACE': 'nierozbawienie',
                  'UPSIDE-DOWN FACE': 'uśmiech',
                  'WEARY FACE': 'zmęczenie',
                  'WINKING FACE': 'mrugnięcie'}

In [ ]:
not_wanted_words = ['@anonymized_account', 'account', 'anonymized' ]

emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U0001F700-\U0001F77F"  # alchemical symbols
          u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
          u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
          u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
          u"\U0001FA00-\U0001FA6F"  # Chess Symbols
          u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
          u"\U00002702-\U000027B0"  # Dingbats
          u"\U000024C2-\U0001F251" 
                            "]+", flags=re.UNICODE)

only_faces_emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
                            "]+", flags=re.UNICODE)

def word_process_with_emoji(word, previous_word=''):
  if word in not_wanted_words:
    return False

  if emoji_pattern.match(word):
    return False
  if only_faces_emoji_pattern.match(word):

    word = unicodedata.name(word)

    if word in emojis_to_words: 
      word = emojis_to_words[word]
    else: return False
    if word == previous_word:
      return False
    return word
  word = word.lower()

  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break 
  word = word.lower()
  if len(word)==0:
    return False
  return word


def clean_dataset_file_emoji():
  dataset_clean = []
  processed_words = {}
  re_split = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))
  with open('polish_corpora.txt', 'r', encoding = 'utf8') as file:
    with open('dataset_clean5.txt', 'a') as f:

      for i, line in enumerate(file):
        if i%10000==0: 
          print(i)
          for item in dataset_clean:
              f.write(item + '\n')

          dataset_clean = []
        if i==5000000: break
        row = []
        words = re_split.split(line)
        if words[0] == 'RT': 
          continue
        new_words = []
        for word in words:
          if len(word) == 0: continue
          if not emoji_pattern.match(word) and not only_faces_emoji_pattern.match(word):
            new_words.append(word)
          elif len(word) == 1:
            new_words.append(word)
          else:
            word = emoji.get_emoji_regexp().split(word)
            word = [i for i in word if len(i)>0]
            new_words += word

        previous_word = ''
        for word in new_words:
          if word in processed_words:
            word = processed_words[word]
          else: 
            new_word = word_process_with_emoji(word, previous_word)
            processed_words[word] = new_word
            word = new_word
          if word != False:
            previous_word = word
          if word != False: 
            row.append(word)
        dataset_clean.append(' '.join(row))
  return dataset_clean, processed_words

dataset_clean, processed_words = clean_dataset_file_emoji()
